# Meteorology-SnowDegreeDay coupling

**Goal:** Try to successfully run a coupled `Meteorology-SnowDegreeDay` simulation. Each component runs to completion in stand-alone mode.

Import the Babel-wrapped `Meteorology` and `SnowDegreeDay` components and create instances:

In [ ]:
from cmt.components import Meteorology, SnowDegreeDay
met, sno = Meteorology(), SnowDegreeDay()

## Demonstrate that the models run independently

Initialize the components with cfg files that, for simplicity, use the same time step and run duration:

In [ ]:
met.initialize('./input/meteorology-2.cfg')
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid):

In [ ]:
time_step = met.get_value('model__time_step').max()
print 'Time step = {} s'.format(time_step)

Advance both models to the end, saving the model time, precipitation flux, and snowmelt flux values at each step:

In [ ]:
time, precip, melt = [], [], []
until = time_step
while met.get_current_time() < met.get_end_time():
    met.update(until)
    time.append(met.get_current_time())
    precip.append(met.get_value('atmosphere_water__precipitation_leq-volume_flux'))
    melt.append(sno.get_value('snowpack__melt_volume_flux'))
    until += time_step

Check the values:

In [ ]:
time

In [ ]:
precip

In [ ]:
melt

Finalize the components:

In [ ]:
met.finalize(), sno.finalize()

**Result:** The models run independently.

## Couple the models

Initialize the components with cfg files that, for simplicity, use the same time step and run duration:

In [ ]:
met.initialize('./input/meteorology-2.cfg')
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid):

In [ ]:
time_step = met.get_value('model__time_step').max()
print 'Time step = {} s'.format(time_step)

Store initial values of time and snow depth:

In [ ]:
time = [met.get_current_time()]
snow_depth = [sno.get_value('snowpack__depth').max()]

Run the coupled models to completion. In each time step, perform the following actions:

1. Advance `SnowDegreeDay`
1. Get variables from `SnowDegreeDay`
1. Pass variables to `Meteorology`
1. Advance `Meteorology`

In [ ]:
until = time_step
while met.get_current_time() < met.get_end_time():
    sno.update(until)
    rho_snow = sno.get_value('snowpack__z_mean_of_mass-per-volume_density')
    h_snow = sno.get_value('snowpack__depth')
    h_swe = sno.get_value('snowpack__liquid-equivalent_depth')
    SM = sno.get_value('snowpack__melt_volume_flux')

    met.set_value('snowpack__z_mean_of_mass-per-volume_density', rho_snow)
    met.set_value('snowpack__depth', h_snow)
    met.set_value('snowpack__liquid-equivalent_depth', h_swe)
    met.set_value('snowpack__melt_volume_flux', SM)
    met.update(until)
    
    time.append(met.get_current_time())
    snow_depth.append(h_snow.max())
    
    until += time_step

In [ ]:
print time

In [ ]:
print snow_depth

Finalize the components:

In [ ]:
met.finalize(), sno.finalize()

**Note:** The Meteorology component was able to advance because of my ugly fix in **BMI_base.py**, line 857, in `set_value`:

```python
if var_name == 'snowpack__depth':
  if self.h_snow.size == self.ny*self.nx:
    self.h_snow = self.h_snow.reshape((self.ny,self.nx))
```

**Result:** Indeterminate. It depends whether my ugly fix is robust.